In [82]:
import numpy as np

import math as m

import itertools

import scipy as sp

In [83]:
def binary_decode(bytearray: list[int]):
    return sum(bytearray * (2 ** np.array(list(enumerate(bytearray.flat)))[:,0]))

def binary_encode(ingest: list[int], meta: list[int]):
    return np.floor_divide(ingest, (2 ** np.array(list(enumerate(meta)))[:,0])) % 2

def func(i):
    return m.tanh(i)

def float_func(i, precision):
    val = i * (2**(precision-1))
    bin_val = binary_encode(val,range(0,(precision)))
    return np.flip(bin_val)

def float_decomp(i, precision):
    exponent = np.arange(0,precision)
    div = np.array([2]*(precision))
    div = div ** exponent
    div = np.array([1]*(precision)) / div
    return np.sum(div*i)

bin_inputs = np.array(list(itertools.product([0, 1], repeat=8)))

bin_inputs[:,0] = bin_inputs[:,0] * 0

inputs = np.apply_along_axis(float_decomp, 1, bin_inputs,precision=8)

index_inputs =  np.array(list(enumerate(inputs.tolist())))[:,0].astype(int)

outputs = np.array([func(i) for i in inputs])

bin_outputs = np.array([float_func(o,8) for o in outputs])

index_outputs = np.array([binary_decode(bo) for bo in bin_outputs]).astype(int)

wildcards = np.vstack([index_inputs,index_outputs]).T
halves = np.array(np.unique(index_outputs,return_counts=True, axis=0))
mixed = np.ones(index_outputs.shape)
mixed[halves[0,:]] = halves[1,:]
superimposed_values = mixed[wildcards[:,1]]

density = sp.sparse.csr_matrix(((1/np.sqrt(superimposed_values)),(index_inputs,index_outputs)),(index_inputs.size,index_outputs.size)).toarray()

input_vectors_matrix = np.diag(np.ones(index_inputs.shape))

output_vectors_matrix = sp.sparse.csr_matrix((np.ones(index_outputs.shape),(index_inputs,index_outputs)),(index_inputs.size,index_outputs.size)).toarray()

unitary = np.linalg.qr(output_vectors_matrix)[0]

In [107]:
precision = 8

input = float_func(0.75, precision)

input_basis_state = np.zeros(2**precision)

input_basis_state[int(binary_decode(input))] = 1

print(input_basis_state.nonzero())

output_basis_state = input_basis_state @ unitary

prob_indexes = output_basis_state.nonzero()

probs = output_basis_state[prob_indexes]

max_prob = np.argmax(output_basis_state)

print(probs)

#bin_output = binary_encode(max_prob,range(0,(precision)))

bin_output = binary_encode(prob_indexes[0][0],range(0,(precision)))

print(bin_output)

float_decomp(bin_output, precision)

(array([6]),)
[-5.00000000e-01 -5.00000000e-01 -4.08248290e-01 -6.56976373e-17
 -3.33333333e-01 -3.33333333e-01  3.33333333e-01]
[1 0 1 1 1 1 1 0]


0.515625

In [85]:
func(0.75)

0.6351489523872873